In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import intake
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats

## Get Observations (HadCRUT4; Morice et al. 2012)

In [ ]:
ds = xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/cru/hadcrut4/air.mon.anom.median.nc').load()
obs = ds['air']
obs

## Create Dask Cluster

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client

cluster = KubeCluster()
cluster.adapt(minimum=2, maximum=100, wait_count=60)
client = Client(cluster)
client

In [ ]:
cluster

## Load data into xarray from an intake catalog

In [ ]:
cat = intake.Catalog('https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/atmosphere/daily.yaml')

ds_20C = cat['reference_height_temperature_20C'].to_dask()
ds_rcp = cat['reference_height_temperature_RCP85'].to_dask()
t_20c = ds_20C['TREFHT']
t_rcp = ds_rcp['TREFHT']

In [ ]:
t_ref = t_20c.sel(time=slice('1961', '1990'))
t_ref

## Compute areacella

Area element: $\delta A = R^2 \delta \phi \delta \lambda \cos(\phi)$

where $\phi$ is latitude, $\delta \phi$ is the spacing of the points in latitude, $\delta \lambda$ is the spacing of the points in longitude, and $R$ is Earth's radius. (In this formula, $\phi$ and $\lambda$ are measured in radians.)

In [ ]:
R = 6.371e6
total_area = (4 * np.pi * R**2)
dϕ = np.radians((t_ref['lat'][1]-t_ref['lat'][0]).values)
dλ = np.radians((t_ref['lon'][1]-t_ref['lon'][0]).values)
dA = R**2 * np.abs(dϕ * dλ) * np.cos(np.radians(t_ref.lat))
areacella = dA * (0 * t_ref.isel(member_id=0, time=0) + 1)
areacella.plot()

In [ ]:
areacella.sum().values / total_area

## Compute Weighted Means

In [ ]:
t_ref_ts = ((t_ref.resample(time='AS').mean('time') * areacella).sum(dim=('lat', 'lon')) / total_area).mean(dim=('time', 'member_id'))
t_20c_ts = ((t_20c.resample(time='AS').mean('time') * areacella).sum(dim=('lat', 'lon'))) / total_area
t_rcp_ts = ((t_rcp.resample(time='AS').mean('time') * areacella).sum(dim=('lat', 'lon'))) / total_area

In [ ]:
t_ref_mean = t_ref_ts.load()
t_ref_mean

In [ ]:
t_20c_ts_df = t_20c_ts.to_series().unstack().T
t_20c_ts_df.head()

In [ ]:
t_rcp_ts_df = t_rcp_ts.to_series().unstack().T
t_rcp_ts_df.head()

- **Obs mean: weight by days in Each Month**

In [ ]:
ds.time_bnds

In [ ]:
time_bound_diff = ds.time_bnds.diff(dim='nbnds')[:, 0]
time_bound_diff

In [ ]:
wgts = time_bound_diff.groupby('time.year') / time_bound_diff.groupby('time.year')\
                      .sum(xr.ALL_DIMS)
wgts

In [ ]:
np.testing.assert_allclose(wgts.groupby('time.year').sum(xr.ALL_DIMS), 1.0)

In [ ]:
cond = obs.isnull()
ones = xr.where(cond, 0.0, 1.0)
obs_sum = (obs * wgts).resample(time='AS').sum(dim='time')
ones_out = (ones * wgts).resample(time='AS').sum(dim='time')
obs_s = (obs_sum / ones_out).mean(('lat', 'lon')).to_series()
obs_s.head()

In [ ]:
all_ts_anom = pd.concat([t_20c_ts_df, t_rcp_ts_df]) - t_ref_mean.data
years = [val.year for val in all_ts_anom.index]

## Confirm that after using area weighted average, max temp increase is 5k

In [ ]:
np.testing.assert_allclose(all_ts_anom.values.max(), 5.0, rtol=0.02)

## Figure 2: Global surface temperature anomaly (1961-90 base period) for individual ensemble members, and observations

In [ ]:
ax = plt.axes()

ax.tick_params(right=True, top=True, direction='out', length=6, width=2, grid_alpha=0.5)
ax.plot(years, all_ts_anom, color='grey')
ax.plot(years, all_ts_anom[1], color='black')
ax.plot(obs_s.index.year.tolist(), obs_s, color='red')

ax.text(0.3, 0.4, 'observations',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color='red', fontsize=10)
ax.text(0.3, 0.3, 'members 1-40',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color='grey', fontsize=10)

ax.set_xticks([1850, 1920, 1950, 2000, 2050, 2100])
plt.ylim(-1, 5)
plt.xlim(1850, 2100)
plt.ylabel('Global Surface\nTemperature Anomaly (K)')
plt.show()

## Compute Linear Trend for Winter Seasons

In [ ]:
def linear_trend(da, dim='time'):
    da_chunk = da.chunk({dim: -1})
    trend = xr.apply_ufunc(calc_slope, da_chunk,
                           vectorize=True,
                           input_core_dims=[[dim]],
                           output_core_dims=[[]],
                           output_dtypes=[np.float],
                           dask='parallelized')
    return trend
    

def calc_slope(y):
    """ufunc to be used by linear_trend"""
    x = np.arange(len(y))
    return np.polyfit(x, y, 1)[0]

In [ ]:
# TODO - this should probably include only full seasons from 1979 and 2012
t = xr.concat([t_20c, t_rcp], dim='time')
seasons = t.sel(time=slice('1979', '2012')).resample(time='QS-DEC').mean('time').load()

In [ ]:
winter_seasons = seasons.sel(time=seasons.time.where(seasons.time.dt.month == 12, drop=True))
winter_trends = linear_trend(winter_seasons.chunk({'lat': 20, 'lon': 20, 'time': -1}))\
                .load() * len(winter_seasons.time)

In [ ]:
len(winter_seasons.time)

In [ ]:
#assert len(winter_seasons.time) == 34  # TODO: this should be 34 I think, its not!

## Figure 4: Global maps of historical (1979 - 2012) boreal winter (DJF) surface air trends

In [ ]:
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(dpi=300)
levels = [-7, -6, -5, -4, -3, -2, -1, -0.5, 0, 0.5, 1, 2, 3, 4, 5, 6, 7]
fg = winter_trends.plot(col='member_id', col_wrap=4, transform=ccrs.PlateCarree(),
            subplot_kws={'projection': ccrs.Robinson(central_longitude=180)},
            add_colorbar=False, levels=levels, cmap='RdYlBu_r', extend='neither')

for ax in fg.axes.flat:
    ax.coastlines(color='grey')
    
# TODO: move the subplot title to lower left corners
# TODO: Add obs panel and ensemble mean at the end
    
fg.add_colorbar(orientation='horizontal')
fg.cbar.set_label('1979-2012 DJF surface air temperature trends (K/34 years)')
fg.cbar.set_ticks(levels)
fg.cbar.set_ticklabels(levels)